In [43]:
import scipy
import numpy as np
import pandas as pd
import pingouin as pg
import matplotlib.pyplot as plt

df = pd.read_csv("football.csv")

Possession, 

npxGD - difference between "for" and "against" expected goals without penalties and own goals

ShortPassesPerGame / LongBallsPerGame

Davandegi

In [44]:
df["Start Season"] = df["Season"].apply(lambda x : int(x[:4]))
df["End Season"] = df["Season"].apply(lambda x : int(x[5:]))
df["League"].replace('La Liga', 'La_liga',inplace = True)
df["League"].replace('Premier League', 'EPL',inplace = True)

In [45]:
df_understat = pd.read_csv("understat.com.csv")
df_understat.rename(columns = {
    "Unnamed: 0" : "league",
    "Unnamed: 1" : "season",
    "position" : "rank"
    }, inplace = True)
pl_u = df_understat[(df_understat["league"] == "EPL") & (df_understat["season"] == 2019)][["rank","team", "npxGD"]].sort_values(["rank"]).reset_index(drop = True)
ll_u = df_understat[(df_understat["league"] == "La_liga") & (df_understat["season"] == 2019)][["rank","team", "npxGD"]].sort_values(["rank"]).reset_index(drop = True)

In [46]:
year = 2019

In [47]:
# SPPLBP - ShortPassesPerGameDividedbyLongBallsPerGame
pl = df[(df["Start Season"] == year) & ((df["League"] == 'EPL') )][['Rank','League','Team','Possession','ShortPassesPerGame', 'LongBallsPerGame']].sort_values(["Rank"]).reset_index(drop = True)
pl["SPPLBP"] = pl["ShortPassesPerGame"] /pl["LongBallsPerGame"]
pl.drop(["LongBallsPerGame", "ShortPassesPerGame"], axis = 1, inplace = True)
pl["Rank"] = pl["Rank"].astype(int)

In [48]:
ll = df[(df["Start Season"] == year) & ((df["League"] == 'La_liga') )][['Rank','League','Team','Possession','ShortPassesPerGame', 'LongBallsPerGame']].sort_values(["Rank"]).reset_index(drop = True)
ll["SPPLBP"] = ll["ShortPassesPerGame"] /ll["LongBallsPerGame"]
ll.drop(["LongBallsPerGame", "ShortPassesPerGame"], axis = 1, inplace = True)
ll["Rank"] = ll["Rank"].astype(int)

In [49]:
pl["npxGD"] = pl_u["npxGD"]
ll["npxGD"] = ll_u["npxGD"]

In [55]:
pl

,Rank,League,Team,Possession,SPPLBP,npxGD
0,1,EPL,Liverpool,59.6,8.590909,32.575792
1,2,EPL,Manchester City,62.6,12.169811,58.818903
2,3,EPL,Manchester United,54.6,9.836735,19.754725
3,4,EPL,Chelsea,57.9,9.807018,31.333631
4,5,EPL,Leicester,55.1,8.600000,15.391501
5,6,EPL,Tottenham,51.5,7.981818,-2.966332
6,7,EPL,Wolverhampton,48.7,5.544118,16.218258
7,8,EPL,Arsenal,52.9,8.377358,-2.468798
8,9,EPL,Sheffield United,44.4,4.364865,-2.954927
9,10,EPL,Burnley,43.9,3.712329,-4.708494


In [56]:
ll

,Rank,League,Team,Possession,SPPLBP,npxGD
0,1,La_liga,Real Madrid,57.1,7.953846,32.321739
1,2,La_liga,Barcelona,63.2,14.170213,34.928789
2,3,La_liga,Atletico Madrid,48.9,6.271186,16.324050
3,4,La_liga,Sevilla,55.9,6.514286,24.196848
4,5,La_liga,Villarreal,29.6,6.523077,15.597880
5,6,La_liga,Real Sociedad,54.6,7.890909,12.003165
6,7,La_liga,Granada,45.2,3.867647,1.581915
7,8,La_liga,Getafe,45.8,2.780488,1.322649
8,9,La_liga,Valencia,49.4,6.262295,-12.754138
9,10,La_liga,Osasuna,47.8,3.986667,-2.771806


In [59]:
pl_ll = pd.concat([pl,ll]).reset_index(drop = True)

In [60]:
pl_ll

,Rank,League,Team,Possession,SPPLBP,npxGD
0,1,EPL,Liverpool,59.6,8.590909,32.575792
1,2,EPL,Manchester City,62.6,12.169811,58.818903
2,3,EPL,Manchester United,54.6,9.836735,19.754725
3,4,EPL,Chelsea,57.9,9.807018,31.333631
4,5,EPL,Leicester,55.1,8.600000,15.391501
5,6,EPL,Tottenham,51.5,7.981818,-2.966332
6,7,EPL,Wolverhampton,48.7,5.544118,16.218258
7,8,EPL,Arsenal,52.9,8.377358,-2.468798
8,9,EPL,Sheffield United,44.4,4.364865,-2.954927
9,10,EPL,Burnley,43.9,3.712329,-4.708494
